<a href="https://colab.research.google.com/github/indhu68/Intro_to_DL_Project/blob/main/RTML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00


In [2]:
!pip install haversine

In [3]:

import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GINConv, global_mean_pool
from itertools import combinations


In [4]:

from google.colab import drive
drive.mount('/content/drive')
# Load data
df = pd.read_csv('/content/drive/My Drive/Kasungu_Telemetry_Pts_Oct23.csv', parse_dates=['Time.Stamp'])
df

Mounted at /content/drive


,Index,Tag,Type,Latitude,Longitude,Time.Stamp,DOP,Speed,Battery,Movement,Alarm,RSSI,Coverage,Retries,SW_Ver_,Time_Stamp,Log_Interv,Temperatur,Accelerome,SW.Ver.
0,10000,5748,IR-SAT Tag,-13.027828,33.133265,2022-07-14 11:08:00,10 meters,3 km/h,3.68 Volt,Y,N,5,N,4,66,7/14/2022,Every 30 minutes,41,"[1033,5505,15905]",66
1,10001,5744,IR-SAT Tag,-13.224110,33.157962,2022-07-14 11:10:00,<10 meters,0 km/h,3.65 Volt,Y,N,4,Y,0,66,7/14/2022,Every 30 minutes,41,"[1222,-960,15876]",66
2,10002,5740,IR-SAT Tag,-13.050495,33.152538,2022-07-14 11:23:00,10 meters,0 km/h,3.68 Volt,Y,N,5,N,2,66,7/14/2022,Every 30 minutes,39,"[-1007,203,15311]",66
3,10005,5746,IR-SAT Tag,-13.061195,33.182005,2022-07-14 11:24:00,10 meters,2 km/h,3.66 Volt,Y,N,3,N,2,66,7/14/2022,Every 30 minutes,41,"[2940,984,14900]",66
4,10006,5745,IR-SAT Tag,-13.007968,33.104750,2022-07-14 11:24:00,10 meters,0 km/h,3.67 Volt,Y,N,5,N,2,66,7/14/2022,Every 30 minutes,40,"[721,-1718,15198]",66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383055,9995,5751,IR-SAT Tag,-12.732593,33.183252,2023-08-16 00:04:00,10 meters,0 km/h,3.66 Volt,Y,N,2,N,3,66,8/16/2023,Every 1 hour,25,"[-441,284,15724]",66
383056,9996,5750,IR-SAT Tag,-12.730062,33.184862,2023-08-16 00:06:00,10 meters,2 km/h,3.64 Volt,Y,N,1,Y,0,66,8/16/2023,Every 1 hour,26,"[-1446,818,15792]",66
383057,9997,5757,IR-SAT Tag,-12.719173,33.082990,2023-08-16 00:12:00,10 meters,0 km/h,3.68 Volt,Y,N,4,Y,0,66,8/16/2023,Every 1 hour,23,"[-14394,1689,6386]",66
383058,9998,5743,IR-SAT Tag,-13.035063,33.221140,2023-08-16 00:17:00,10 meters,0 km/h,3.69 Volt,Y,N,5,Y,0,66,8/16/2023,Every 1 hour,22,"[2096,3363,15104]",66


In [5]:
data = df[["Tag", "Latitude", "Longitude", "Time.Stamp"]]

In [6]:
df['Time.Stamp'] = pd.to_datetime(df['Time.Stamp'])
df.sort_values(by=['Tag', 'Time.Stamp'], inplace=True)

In [7]:
# Drop rows with NaN values
data = data.dropna(subset=["Latitude", "Longitude", "Time.Stamp"])

In [8]:
from haversine import haversine as haversine_dist


def get_speed_direction(data):
    speeds = np.zeros(len(data))
    directions = np.zeros(len(data))

    for i in range(1, len(data)):
        # Calculate speed and direction for each row
        speed = np.sqrt((data["Latitude"][i] - data["Latitude"][i-1])**2 + (data["Longitude"][i] - data["Longitude"][i-1])**2)
        direction = np.arctan2((data["Latitude"][i] - data["Latitude"][i-1]), (data["Longitude"][i] - data["Longitude"][i-1]))

        speeds[i] = speed
        directions[i] = direction

    return speeds, directions

# Assuming data is your DataFrame
data["Speed"], data["Direction"] = get_speed_direction(data)




In [9]:
data

,Tag,Latitude,Longitude,Time.Stamp,Speed,Direction
0,5748,-13.027828,33.133265,2022-07-14 11:08:00,0.000000,0.000000
1,5744,-13.224110,33.157962,2022-07-14 11:10:00,0.197830,-1.445630
2,5740,-13.050495,33.152538,2022-07-14 11:23:00,0.173700,1.602028
3,5746,-13.061195,33.182005,2022-07-14 11:24:00,0.031350,-0.348313
4,5745,-13.007968,33.104750,2022-07-14 11:24:00,0.093816,2.538302
...,...,...,...,...,...,...
383055,5751,-12.732593,33.183252,2023-08-16 00:04:00,0.148416,2.148143
383056,5750,-12.730062,33.184862,2023-08-16 00:06:00,0.003000,1.004246
383057,5757,-12.719173,33.082990,2023-08-16 00:12:00,0.102452,3.035108
383058,5743,-13.035063,33.221140,2023-08-16 00:17:00,0.344778,-1.158524


In [10]:
from sklearn.preprocessing import StandardScaler

# Assuming 'data' is your DataFrame containing 'Latitude', 'Longitude', 'Speed', and 'Direction' columns
columns_to_scale = ['Latitude', 'Longitude', 'Speed', 'Direction']
scaler = StandardScaler()
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

In [11]:
# Split data into training and testing sets
train_data = data.iloc[:-int(0.2 * len(data))]  # 80% for training
val_data = data.iloc[-int(0.2 * len(data)):]

In [12]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

# Define a simple LSTM RNN model
class ElephantMovementRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(ElephantMovementRNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 2)  # 2 for latitude and longitude prediction

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(0)  # Add batch dimension

        x, _ = self.lstm(x)  # LSTM output and hidden state (not used here)
        out = self.linear(x[:, -1, :])  # Use output from the last timestep
        return out

# Prepare training and validation data (tensors)
train_x = torch.tensor(train_data[["Speed", "Direction"]].values, dtype=torch.float)
train_y = torch.tensor(train_data[["Latitude", "Longitude"]].values, dtype=torch.float)
val_x = torch.tensor(val_data[["Speed", "Direction"]].values, dtype=torch.float)
val_y = torch.tensor(val_data[["Latitude", "Longitude"]].values, dtype=torch.float)

train_dataset = TensorDataset(train_x, train_y)
val_dataset = TensorDataset(val_x, val_y)

# Define dataloaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Model hyperparameters
model = ElephantMovementRNN(2, 64, 2)  # Input size, hidden size, num layers
criterion = nn.MSELoss()  # Mean squared error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the RNN model without early stopping
num_epochs = 100

for epoch in range(num_epochs):
    # Training loop (mentioned previously)
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation after each epoch
    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels)
        avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss}")




/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16, 2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4, 2])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(i

Epoch 1, Validation Loss: 0.6272197961807251
Epoch 2, Validation Loss: 0.6241517066955566
Epoch 3, Validation Loss: 0.6216321587562561
Epoch 4, Validation Loss: 0.6287431716918945
Epoch 5, Validation Loss: 0.6325986385345459
Epoch 6, Validation Loss: 0.6252233386039734
Epoch 7, Validation Loss: 0.6269780993461609
Epoch 8, Validation Loss: 0.6347459554672241
Epoch 9, Validation Loss: 0.6312077641487122
Epoch 10, Validation Loss: 0.6274827718734741
Epoch 11, Validation Loss: 0.6225918531417847
Epoch 12, Validation Loss: 0.6231635808944702
Epoch 13, Validation Loss: 0.6216944456100464
Epoch 14, Validation Loss: 0.6208987236022949
Epoch 15, Validation Loss: 0.63127601146698
Epoch 16, Validation Loss: 0.6224950551986694
Epoch 17, Validation Loss: 0.6227447390556335
Epoch 18, Validation Loss: 0.6205831170082092
Epoch 19, Validation Loss: 0.622891902923584
Epoch 20, Validation Loss: 0.6226024627685547
Epoch 21, Validation Loss: 0.6289066076278687
Epoch 22, Validation Loss: 0.617848813533783
E

In [13]:
# Test data (tensor)
test_x = torch.tensor(val_data[["Speed", "Direction"]].values, dtype=torch.float)

# Predict on test data using the best model
with torch.no_grad():
  predicted_locations = model(test_x)

# Convert predicted tensors back to NumPy arrays for further processing/visualization
predicted_lats = predicted_locations[:, 0].numpy()
predicted_lons = predicted_locations[:, 1].numpy()